# This script processes SYCL and CUDA results 

1. Go to SYCL-CUDA directory and compile all CPU versions with:
```
make owm-sycl-cpu owm-sycl-cpu-nomemo
```
2. Run the benchmarking script
```
cd scripts
python run_sycl_cpu_speedup.py
```
3. The output that we will process is `sycl_cpu_speedup_<hostname>.txt` that should be already in the `Results` directory.



In [ ]:
import os
# get the hostname of the server
hostname = os.popen("hostname").read().strip()
# ensure the directory exists
os.makedirs(hostname, exist_ok=True)
# ensure the file with baseline results exists
sycl_file = f'sycl_cpu_speedup_{hostname}.txt'
# if it is not in Results
if not os.path.exists(sycl_file):
    # is the file already in the directory?
    assert os.path.exists(os.path.join(hostname, sycl_file)), f'File {sycl_file} not found: something went wrong with the sycl_cuda benchmark.'
# if it is in Results
else:
    # copy the file to the directory
    assert os.system(f'mv {sycl_file} {hostname}/') == 0, f'Failed to move {sycl_file} to {hostname}/'
# rename sycl_file
sycl_file = os.path.join(hostname, sycl_file)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from common.utils import tokenize_sycl_cpu

res_cpu = tokenize_sycl_cpu(sycl_file)

In [ ]:
#Configuration variables
def plot_res(res):
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8

    fig = plt.figure()

    marks=['o-','x-','s-','v-','+-']
    # there is only one possible level for each cloud, as we performed the tests with only one level
    bestmN=[min(res[i], key=lambda x: res[i][x][8][4]) for i in res]
    numthreads=list(res['Alcoy'][bestmN[0]].keys())
    print(bestmN,numthreads)
    labels=['Tree Const.','OWM Trav.','Total']

    fig, axs = plt.subplots(nrows=1, ncols=3,figsize=(15, 5), constrained_layout=True)
    sizes=[i+" (mN="+str(j)+")" for i,j in zip(res,bestmN)]
    sizes.append('Ideal')

    for subfig,phase in enumerate([0,3,4]): # ['qtree','owm','total']
        # print("Phase: "+phase+ " speedup for each cloud and best level for each cloud: "+str(list(res.keys())))
        for (i,z,mN) in zip(res,marks,bestmN):
            # print(np.array([res[i][mN][1][subfig]/res[i][mN][j][subfig] for j in numthreads]))
            axs[subfig].plot(np.array(numthreads), np.array([res[i][mN][1][phase]/res[i][mN][j][phase] for j in numthreads]), z, linewidth=linew, markersize=markers)
        axs[subfig].plot(np.array(numthreads), np.array(numthreads), '-', linewidth=linew, markersize=markers)
        axs[subfig].set_title(f'{labels[subfig]}',  fontweight='bold', fontsize=ylabelfs)
        axs[subfig].set_xlabel('Number of cores', fontsize=xlabelfs)
        axs[subfig].set_xticks(numthreads)
        axs[subfig].tick_params(axis='x', labelsize=xticksfs)
        axs[subfig].grid()

    fig.suptitle(f'Speedup for each cloud and each phase SYCL-CPU @ {hostname}', fontweight='bold', fontsize=titlefs)
    axs[0].set_ylabel('Speedup', fontsize=ylabelfs)
    axs[0].legend(sizes,loc='best', fontsize= legendfs)

    return fig

fig = plot_res(res_cpu['owm-sycl-cpu'])

In [ ]:
pp = PdfPages(f"{hostname}/Speedup-SYCL-CPU-{hostname}.pdf")
pp.savefig(fig)
pp.close()